# One-way ANOVA test and pairwise comparisons with TukeyHSD confidence intervals

In [ ]:
# imports and setting matplotlib style
from collections import OrderedDict
import numpy as np
import scipy
import scipy.stats
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd
plt.style.use('ggplot')
#print(plt.style.available)
%matplotlib inline

In [ ]:
import json
#TRYING TO CHANGE SOMETHING HERE
seek_f = open('./seek-notebook-data-json20180528-26582-yazu7p.json')
# now read the file
seek_f_content = seek_f.read();
seek_f_json = json.loads(seek_f_content, object_pairs_hook=OrderedDict);

# print for debugging purposes. Also interesting for the user, so leave in.
# print(seek_f_json)


In [ ]:
marked = seek_f_json["marked"];

# https://stackoverflow.com/questions/354038/how-do-i-check-if-a-string-is-a-number-float
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

# put the right keys into a file that 
def headlinify(j):
    retval = OrderedDict()
    for (k,v) in j.items():
        first = v["values"][0].encode('utf-8');
        effective_key = "";
        if(not is_number(first)):
            retval[first] = v["values"][1:]
            effective_key = first 
        else:
            retval[k] = v;
            effective_key = k;
            
        retval[effective_key] = list(map(float,filter(is_number,retval[effective_key])))
            
    return retval
        
    
data_sep = headlinify(marked);


In [ ]:
#print(data_sep)
labels = []
for lab in data_sep.keys():
    labels.append(lab.decode('utf-8'))
print labels

In [ ]:
# histogram of data for covenience
n, bins, patches = plt.hist(data_sep.values(), int(sum(map(len, data_sep.values()))/4), histtype='stepfilled', alpha=0.75, label=labels)
plt.legend()
plt.show()

In [ ]:
# box plots of data for covenience
plt.boxplot(data_sep.values(), labels=labels)
plt.violinplot(data_sep.values())
plt.savefig("boxplots.png")
plt.show()

# Results of anova and pairwise Tukey HSD

In [ ]:
anova_results = scipy.stats.f_oneway(*data_sep.values())
import pandas as pd
df = pd.DataFrame(data=[anova_results[0],anova_results[1]], index=['F-value', 'p-value'], columns=['1-way ANOVA'])

In [ ]:
data_sep.values();

In [ ]:
def structure_for_tukeyhsd(sep):
    values = []
    groups = []
    for k,l in sep.items():
        for x in l:
            values.append(x)
            groups.append(k)
    values = np.array(values)
    groups = np.array(groups)
    return (values,groups)
            
(values,groups) = structure_for_tukeyhsd(data_sep)

In [ ]:
#from statsmodels.stats.multicomp import MultiComparison
#mc = MultiComparison(values, groups)
#result = mc.tukeyhsd()
#print(result)
#print(mc.groupsunique)
results = pairwise_tukeyhsd(values,groups)
fig = results.plot_simultaneous()

In [ ]:
results.summary()

In [ ]:
df